# Video to Image
before style transfer using AesFA

In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import glob

resolution = '2048'
content_dir = './video/{}/'.format(resolution)
contents_in = sorted(glob.glob(content_dir+'*.mp4'))
content_img = './video_img/{}/content/'.format(resolution)
if not os.path.isdir(content_img):
    os.makedirs(content_img)

contents = []
for content in contents_in:
    content_name = content.split('/')[-1].split('.')[0]
    contents.append(content_name)
    print(content_name)
    
    # Video Capture Sanity Check
    cap = cv2.VideoCapture(content)
    length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    print(length, fps)

    i = 0
    while(cap.isOpened()):
        ret, frame = cap.read()
        try:
            frame = frame[:,:,::-1]
            iframe = Image.fromarray(frame)

            iframe.save(content_img+'{:s}_{:d}.jpg'.format(content_name, i+1))
            i += 1
        except:
            break

    cap.release()

pexels-kelly-7165765-3840x2160-24fps
319 24.0


# Transfer the style of each frame using AesFA model

In [2]:
# Before testing, you have to edit Config.py file as you want.
# content_dir would be like './video_img/2048/content/'
# img_dir would be like './output/video_output/2048'
import test_video
test_video.main()

Version: main
cuda:0
checkpoint:  ./ckpt/main/main.pth
content dir: ./video_img/2048/content/
style dir: /global/cfs/cdirs/m3898/GANBERT/st_to_diff/Code/LabServer/ResolutionDataset/2048/style
# of contents: 319
# of styles: 2
1 1 1 1.4959888458251953
2 1 2 0.013611078262329102
3 2 1 0.013701677322387695
4 2 2 0.013676643371582031
5 3 1 0.013656854629516602
6 3 2 0.013533353805541992
7 4 1 0.01367640495300293
8 4 2 0.013755083084106445
9 5 1 0.013662338256835938
10 5 2 0.01371622085571289
11 6 1 0.01367497444152832
12 6 2 0.013765335083007812
13 7 1 0.011701345443725586
14 7 2 0.011647224426269531
15 8 1 0.01172637939453125
16 8 2 0.01176309585571289
17 9 1 0.011693477630615234
18 9 2 0.011683225631713867
19 10 1 0.012634038925170898
20 10 2 0.011624336242675781
21 11 1 0.011622428894042969
22 11 2 0.01162266731262207
23 12 1 0.01264333724975586
24 12 2 0.012636184692382812
25 13 1 0.011599302291870117
26 13 2 0.011915445327758789
27 14 1 0.01216435432434082
28 14 2 0.0116424560546875
2

# Image to Video - Content Video
After style transfer

In [3]:
import cv2
import numpy as np
import glob
from Config import Config

config = Config()
out_dir = config.img_dir

for content_n in contents:
    length = len(glob.glob(out_dir+'/content/'+content_n+'*_content*'))
    cap = cv2.VideoCapture(content_dir+content_n+'.mp4')
    fps = cap.get(cv2.CAP_PROP_FPS)
    print(length, fps)
    
    img_array = []
    for idx in range(length):
        filename = out_dir+'/content/'+content_n+'_'+str(idx+1)+'_content.jpg'
        img = cv2.imread(filename)
        img_array.append(img)
    height, width, layers = img.shape
    size = (width,height)

    out = cv2.VideoWriter(out_dir+'/CONTENT_'+content_n+'.mp4', cv2.VideoWriter_fourcc(*'MP4V'), fps, size)

    for i in range(len(img_array)):
        out.write(img_array[i])
    out.release()

319 24.0


OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


# Image to Video - Style Transfered Video

In [4]:
style_imgs = sorted(glob.glob('./video_img/'+resolution+'/style/*.jpg'))

for content in contents:
    length = len(glob.glob(out_dir+'/content/'+content+'*_content*'))
    cap = cv2.VideoCapture(content_dir+content+'.mp4')
    fps = cap.get(cv2.CAP_PROP_FPS)
    print(content, length, fps)
    
    for style_dir in style_imgs:
        style = style_dir.split('/')[-1].split('.')[0]
        img_array = []

        for idx in range(length):
            filename = out_dir+'/stylized/'+content+'_'+str(idx+1)+'_stylized_'+style+'.jpg'
            img = cv2.imread(filename)
            img_array.append(img)
        height, width, layers = img.shape
        size = (width,height)
        print("-", style, len(img_array))

        out = cv2.VideoWriter(out_dir+'/STYLE_'+content+'_'+style+'.mp4', cv2.VideoWriter_fourcc(*'MP4V'), fps, size)

        for i in range(len(img_array)):
            out.write(img_array[i])
        out.release()

pexels-kelly-7165765-3840x2160-24fps 319 24.0
